In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/Raisin_Dataset.csv"
raisin = pd.read_csv(file_path, index_col = 0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
raisin.head()

,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
Area,,,,,,,
87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


In [5]:
raisin.reset_index(inplace = True)
raisin.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


In [29]:
raisin['Class'] = raisin['Class'].replace({'Kecimen': 1, 'Besni': 0})

print(raisin['Class'])

0      0
1      0
2      0
3      0
4      0
      ..
895    1
896    1
897    1
898    1
899    1
Name: Class, Length: 900, dtype: int64


In [31]:
raisin_input  = raisin[["Area", "MajorAxisLength", "MinorAxisLength", "Eccentricity", "ConvexArea", "Extent", "Perimeter"]].to_numpy()
raisin_target = raisin["Class"].to_numpy()

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

train_input, test_input, train_target, test_target = train_test_split(raisin_input, raisin_target, test_size = 0.25)

ss = StandardScaler()
ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled  = ss.transform(test_input)

In [33]:
import numpy as np

train_target = np.array(train_target)
test_target  = np.array(test_target)

In [34]:
print(train_scaled.shape)
print(test_scaled.shape)

(675, 7)
(225, 7)


In [35]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow._api.v2.config import optimizer

model = keras.Sequential()
model.add(keras.layers.Dense(units = 2))
model.add(keras.layers.Conv2D(64, kernel_size = 2, activation = 'relu', padding = 'same', input_shape = (train_scaled.shape[1], 1)))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.MaxPooling1D(2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units = 16, activation = 'relu'))

In [38]:
from sklearn.utils import validation


model.compile(loss = "sparse_categorical_crossentropy",
                metrics = "accuracy",
                optimizer = "adam")
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True, mode='auto')

history = model.fit(train_scaled, train_target,
                        epochs = 100,
                        verbose = False,
                        validation_data = (test_scaled, test_target),
                        callbacks = [early_stopping_cb, checkpoint_cb])

model.evaluate(test_scaled, test_target)
final_accuracy = result[1]


ValueError: ignored